In [1]:
from datetime import datetime

from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.utils.operators import QubitPauliOperator
from pytket.partition import measurement_reduction,MeasurementBitMap, MeasurementSetup, PauliPartitionStrat
from pytket.backends.backendresult import BackendResult

from pytket.utils.operators import QubitPauliOperator
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit

from scipy.optimize import minimize
from numpy import ndarray
from numpy.random import random_sample
from sympy import Symbol

import qnexus as qnx
from qnexus.references import CircuitRef


# Example VQE workflow using Quantinuum Nexus


VQE example adapted from https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_variational_experiment_with_batching.ipynb

## Set up the VQE components

In [2]:
# 1. Synthesise Symbolic State-Preparation Circuit (hardware efficient ansatz)

symbols = [Symbol(f"p{i}") for i in range(4)]
symbolic_circuit = Circuit(2)
symbolic_circuit.X(0)
symbolic_circuit.Ry(symbols[0], 0).Ry(symbols[1], 1)
symbolic_circuit.CX(0, 1)
symbolic_circuit.Ry(symbols[2], 0).Ry(symbols[3], 0)

#render_circuit_jupyter(symbolic_circuit)

[X q[0]; Ry(p1) q[1]; Ry(p0) q[0]; CX q[0], q[1]; Ry(p2) q[0]; Ry(p3) q[0]; ]

In [3]:
# 2. Define Hamiltonian 
# coefficients in the Hamiltonian are obtained from PhysRevX.6.031007

coeffs = [-0.4804, 0.3435, -0.4347, 0.5716, 0.0910, 0.0910]
term0 = {
    QubitPauliString(
        {
            Qubit(0): Pauli.I,
            Qubit(1): Pauli.I,
        }
    ): coeffs[0]
}
term1 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.I}): coeffs[1]}
term2 = {QubitPauliString({Qubit(0): Pauli.I, Qubit(1): Pauli.Z}): coeffs[2]}
term3 = {QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.Z}): coeffs[3]}
term4 = {QubitPauliString({Qubit(0): Pauli.X, Qubit(1): Pauli.X}): coeffs[4]}
term5 = {QubitPauliString({Qubit(0): Pauli.Y, Qubit(1): Pauli.Y}): coeffs[5]}
term_sum = {}
term_sum.update(term0)
term_sum.update(term1)
term_sum.update(term2)
term_sum.update(term3)
term_sum.update(term4)
term_sum.update(term5)
hamiltonian = QubitPauliOperator(term_sum)



In [4]:
# 3 Computing Expectation Values

# Computing Expectation Values for Pauli-Strings
def compute_expectation_paulistring(
    distribution: dict[tuple[int, ...], float], bitmap: MeasurementBitMap
) -> float:
    value = 0
    for bitstring, probability in distribution.items():
        value += probability * (sum(bitstring[i] for i in bitmap.bits) % 2)
    return ((-1) ** bitmap.invert) * (-2 * value + 1)



# 3.2 Computing Expectation Values for sums of Pauli-strings multiplied by coefficients
def compute_expectation_value(
    results: list[BackendResult],
    measurement_setup: MeasurementSetup,
    operator: QubitPauliOperator,
) -> float:
    energy = 0
    for pauli_string, bitmaps in measurement_setup.results.items():
        string_coeff = operator.get(pauli_string, 0.0)
        if string_coeff > 0:
            for bm in bitmaps:
                index = bm.circ_index
                distribution = results[index].get_distribution()
                value = compute_expectation_paulistring(distribution, bm)
                energy += complex(value * string_coeff).real
    return energy

In [5]:
# 4. Building our Objective function

class Objective:
    def __init__(
        self,
        symbolic_circuit: CircuitRef,
        problem_hamiltonian: QubitPauliOperator,
        n_shots_per_circuit: int,
        target: qnx.BackendConfig,
        iteration_number: int = 0,
        n_iterations: int = 10,
    ) -> None:
        r"""Returns the objective function needed for a variational
        procedure.
        """
        terms = [term for term in problem_hamiltonian._dict.keys()]
        self._symbolic_circuit: Circuit = symbolic_circuit.download_circuit()
        self._hamiltonian: QubitPauliOperator = problem_hamiltonian
        self._nshots: int = n_shots_per_circuit
        self._measurement_setup: MeasurementSetup = measurement_reduction(
            terms, strat=PauliPartitionStrat.CommutingSets
        )
        self._iteration_number: int = iteration_number
        self._niters: int = n_iterations
        self._target = target


    def __call__(self, parameter: ndarray) -> float:
        value = self._objective_function(parameter)
        self._iteration_number += 1
        if self._iteration_number >= self._niters:
            self._iteration_number = 0
        return value
    
    def _objective_function(
        self,
        parameters: ndarray,
    ) -> float:
        assert len(parameters) == len(self._symbolic_circuit.free_symbols())
        circuit_list = self._build_circuits(parameters)
        
        # Execute circuits with Nexus
        execute_job_ref = qnx.execute(
            name=f"execute_job_VQE_{datetime.now()}_{self._iteration_number}",
            circuits=circuit_list,
            n_shots=[self._nshots]*len(circuit_list),
            target=self._target,
        )
        
        qnx.job.wait_for(execute_job_ref)
        results = [item.download_result() for item in qnx.job.results(execute_job_ref)]
        
        expval = compute_expectation_value(
            results, self._measurement_setup, self._hamiltonian
        )
        return expval
    
    def _build_circuits(self, parameters: ndarray) -> list[CircuitRef]:
        circuit = self._symbolic_circuit.copy()
        symbol_dict = {s: p for s, p in zip(self._symbolic_circuit.free_symbols(), parameters)}
        circuit.symbol_substitution(symbol_dict)

        properties = {str(sym): val for sym, val in symbol_dict.items()} | {"iteration": self._iteration_number}

        with qnx.context.using_properties(**properties):
        
            # Upload the numerical state-prep circuit to Nexus
            qnx.circuit.upload(
                circuit=circuit,
                name=f"state prep circuit {self._iteration_number}",
            )
            circuit_list = []
            for mc in self._measurement_setup.measurement_circs:
                c = circuit.copy()
                c.append(mc)
                # Upload each measurement circuit to Nexus with correct params
                measurement_circuit_ref = qnx.circuit.upload(
                    circuit=c, 
                    name=f"state prep circuit {self._iteration_number}",
                )
                circuit_list.append(measurement_circuit_ref)
                
            # Compile circuits with Nexus
            compile_job_ref = qnx.compile(
                name=f"compile_job_VQE_{datetime.now()}_{self._iteration_number}",
                circuits=circuit_list,
                optimisation_level=2,
                target=self._target,
            )
            qnx.job.wait_for(compile_job_ref)
            compile_job_refs = qnx.job.results(compile_job_ref)
            compiled_circuit_refs = [item.get_output() for item in compile_job_refs]

            for ref in compiled_circuit_refs:
                # Update with properties from context
                qnx.circuit.update(ref)
        return compiled_circuit_refs



## Set up the Nexus Project and run the VQE

In [6]:
# set up the project
project_ref = qnx.project.create(
    name=f"VQE_example_{str(datetime.now())}",
    description="A VQE done with qnexus",
)

# set this in the context
qnx.context.set_active_project(project_ref)

<Token var=<ContextVar name='qnexus_project' default=None at 0x17f91aac0> at 0x305aef200>

In [7]:
# Set up the properties
qnx.project.add_property(
    name="iteration", 
    property_type="int", 
    description="The iteration number in my dihydrogen VQE experiment", 
)

for sym in symbolic_circuit.free_symbols():
    qnx.project.add_property(
        name=str(sym), 
        property_type="float",
        description=f"Our VQE {str(sym)} parameter", 
    )

In [8]:
# Upload our ansatz circuit

ansatz_ref = qnx.circuit.upload(
    circuit=symbolic_circuit,
    name="ansatz_circuit",
    description="The ansatz state-prep circuit for my dihydrogen VQE",
)

In [9]:
objective = Objective(
    symbolic_circuit = ansatz_ref,
    problem_hamiltonian = hamiltonian,
    n_shots_per_circuit = 500,
    n_iterations= 4,
    target = qnx.QuantinuumConfig(device_name="H1-1LE")
)

In [10]:
initial_parameters = random_sample(len(symbolic_circuit.free_symbols()))

result = minimize(
    objective,
    initial_parameters,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)

print(result.fun)
print(result.x)

-0.8963936000000001
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    4   F =-8.963936E-01    MAXCV = 0.000000E+00
   X = 1.391201E+00   1.167922E-01   5.091653E-01   9.919147E-01

[1.39120149 0.11679222 0.50916527 0.99191466]


# Use Nexus to Rescue a VQE workflow

For instance, lets say that some failure happened on the 2nd iteration (e.g. laptop ran out of battery) and we want to resume ASAP.

In the above we ran for 4 iterations, lets pretend that we actually wanted to run for 7 and it failed on the 4th one.

N.B. Assuming the minimizer has no internal state (or has been pickled).

In [11]:
# Get the project, fetching the latest one with the name prefix from above
project_matches = qnx.project.get(name_like="VQE_example_", sort=['-timestamps.created'])

project_ref = project_matches.list()[0]

# set this in the context
qnx.context.set_active_project(project_ref)

project_ref.df()

,name,description,created,modified,contents_modified,id
0,VQE_example_2024-05-20 15:18:27.678060,A VQE done with qnexus,2024-05-20 14:18:29.036726+00:00,2024-05-20 14:18:29.036726+00:00,2024-05-20 14:21:59.630404+00:00,b64c33d0-331e-4adc-beb6-749e351f235b


In [12]:
# Get the symbolic circuit
symbolic_circuit_ref = qnx.circuit.get_only(name_like="ansatz_circuit")

In [13]:
most_recent_circuits = qnx.circuit.get(name_like="final", project_ref=project_ref)

most_recent_circuits.summarize()

,resource,total_count
0,Circuit,8


In [14]:
most_recent_circuits_refs = most_recent_circuits.list()

most_recent_circuits_refs.df()

,name,description,created,modified,iteration,p0,p1,p2,p3,project,id
0,state prep circuit 0-QuantinuumBackend-final,None,2024-05-20 14:18:40.885144+00:00,2024-05-20 14:18:40.885144+00:00,0,0.116792,0.991915,0.509165,0.391201,VQE_example_2024-05-20 15:18:27.678060,1ca00e19-61b5-488e-be26-cc508fb0c1e3
1,state prep circuit 0-QuantinuumBackend-final,None,2024-05-20 14:18:42.000433+00:00,2024-05-20 14:18:42.000433+00:00,0,0.116792,0.991915,0.509165,0.391201,VQE_example_2024-05-20 15:18:27.678060,29e02611-0672-4149-b30e-b3b394c9b6a7
2,state prep circuit 1-QuantinuumBackend-final,None,2024-05-20 14:20:54.694704+00:00,2024-05-20 14:20:54.694704+00:00,1,0.116792,0.991915,0.509165,1.391201,VQE_example_2024-05-20 15:18:27.678060,a8e69efc-7359-4c06-88f0-338d860739bf
3,state prep circuit 1-QuantinuumBackend-final,None,2024-05-20 14:20:55.072415+00:00,2024-05-20 14:20:55.072415+00:00,1,0.116792,0.991915,0.509165,1.391201,VQE_example_2024-05-20 15:18:27.678060,ff90fc7d-615b-4507-b064-03f79f554c05
4,state prep circuit 2-QuantinuumBackend-final,None,2024-05-20 14:21:19.156627+00:00,2024-05-20 14:21:19.156627+00:00,2,1.116792,0.991915,0.509165,1.391201,VQE_example_2024-05-20 15:18:27.678060,4c703142-0915-48fd-99ea-15c4e4f225fc
5,state prep circuit 2-QuantinuumBackend-final,None,2024-05-20 14:21:19.288470+00:00,2024-05-20 14:21:19.288470+00:00,2,1.116792,0.991915,0.509165,1.391201,VQE_example_2024-05-20 15:18:27.678060,764e8535-ef33-4fb8-937c-1d0394327cc1
6,state prep circuit 3-QuantinuumBackend-final,None,2024-05-20 14:21:43.323471+00:00,2024-05-20 14:21:43.323471+00:00,3,0.116792,0.991915,1.509165,1.391201,VQE_example_2024-05-20 15:18:27.678060,21aa0d1a-0634-4388-a51e-3759ae5b82db
7,state prep circuit 3-QuantinuumBackend-final,None,2024-05-20 14:21:43.902913+00:00,2024-05-20 14:21:43.902913+00:00,3,0.116792,0.991915,1.509165,1.391201,VQE_example_2024-05-20 15:18:27.678060,9ac64355-a05f-45d3-bb36-0dae78929362


In [15]:
# Get the latest circuit to get the new 'initial_parameters'
latest_circuit: CircuitRef = most_recent_circuits_refs[-1]

latest_circuit_properties = latest_circuit.annotations.properties

latest_circuit.df()

,name,description,created,modified,iteration,p0,p1,p2,p3,project,id
0,state prep circuit 3-QuantinuumBackend-final,None,2024-05-20 14:21:43.902913+00:00,2024-05-20 14:21:43.902913+00:00,3,0.116792,0.991915,1.509165,1.391201,VQE_example_2024-05-20 15:18:27.678060,9ac64355-a05f-45d3-bb36-0dae78929362


In [16]:
# Get what iteration we were on (from the latest circuit)

last_iteration_count = latest_circuit_properties.pop("iteration")

print(last_iteration_count)

# Retreive the params and check them
new_starting_params = list(latest_circuit_properties.values())
print(new_starting_params)


3
[0.1167922168970108, 0.9919146299362183, 1.5091652870178223, 1.3912014961242676]


In [17]:
# Build the Objective and run 'minimize' to continue the experiment
objective = Objective(
    symbolic_circuit_ref,
    hamiltonian,
    n_shots_per_circuit = 500,
    iteration_number=last_iteration_count, # resume from 3rd iteration of 7
    n_iterations = 7,
    target = qnx.QuantinuumConfig(device_name="H1-1LE")
)

result = minimize(
    objective,
    new_starting_params,
    method="COBYLA",
    options={"disp": True, "maxiter": objective._niters},
    tol=1e-2,
)

print(result.fun)
print(result.x)

-0.26560800000000007
   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =    7   F =-2.656080E-01    MAXCV = 0.000000E+00
   X = 1.372074E+00   6.521018E-01   1.247782E+00   1.358993E+00

[1.37207372 0.65210181 1.24778204 1.35899286]
